In [5]:
import pandas as pd
from utils_analysis import prepare_scores
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import json

temp = json.load(
    open("results/bart/r3/bart-base_data_flores_checkpoint-1000_flores.json", "r")
)

In [8]:
df = pd.read_csv("data/flores/test.csv")

# "results/bart/r1/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r2/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r3/bart-base_data_flores_checkpoint-1000_flores.json",
# "results/t5/r1/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r2/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r3/flan-t5-base_data_flores_checkpoint-1000_flores.json",

df_scores, beam_search_results = prepare_scores(
    "results/bart/r3/bart-base_data_flores_checkpoint-1000_flores.json",
    df["target"],
    "bleu",
)

/home/mila/f/floresl/beam-search/utils_analysis.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [9]:
df_scores.drop(["sentences", "dropout_sentences"], axis=1).corr(method="spearman")[
    "bleu"
]

length_normalized_log_probs    0.264683
mean_token_entropy            -0.201433
dropout_bleu_variance          0.230362
dropout_meteor_score           0.474040
dropout_entropy               -0.307667
dropout_disagreement           0.050713
bleu                           1.000000
beam_score_ratios_99           0.076925
beam_score_log_probs_99        0.251105
beam_score_top_k_99            0.262102
beam_score_impt_wt_99         -0.262175
Name: bleu, dtype: float64

In [10]:
beam_score_log_probs = pd.DataFrame(beam_search_results["beam_score_log_probs"])
worst_rouge_idxs = df_scores.sort_values(by="bleu").index[:100]
best_rouge_idxs = df_scores.sort_values(by="bleu").index[::-1][:100]

In [11]:
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr

In [12]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=1)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, df_scores["bleu"])

SignificanceResult(statistic=0.08024942468222501, pvalue=0.010653755454480881)

In [13]:
import json

baseline_probs = json.load(
    open(
        "/home/mila/f/floresl/beam-search/results/bart_baseline/facebook_bart-base_flores.json",
        "r",
    )
)

In [14]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]

In [26]:
k = 5

In [27]:
spearmanr(
    [a - b for (a, b) in zip(beam_score_log_probs[str(k)], baseline_log_probs[str(k)])],
    df_scores["bleu"],
)

SignificanceResult(statistic=0.03521542197779425, pvalue=0.2630404162204451)